In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from google.colab import files

files.upload() #this will prompt you to upload the kaggle.json

KeyboardInterrupt: ignored

In [ ]:
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!pwd
!kaggle datasets list

-rw-r--r-- 1 root root 69 Aug 21 12:24 kaggle.json
/content
ref                                                    title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
deepcontractor/mcdonalds-india-menu-nutrition-facts    McDonald's India : Menu Nutrition Dataset          5KB  2022-07-28 17:08:52           2085        107  0.9411765        
iamsouravbanerjee/house-rent-prediction-dataset        House Rent Prediction Dataset                     82KB  2022-08-20 13:49:03           2094         91  1.0              
arianazmoudeh/airbnbopendata                           Airbnb Open Data                                  10MB  2022-08-01 15:58:10           1495         52  0.9705882        
ericsims/world-cheese-awards-worlds-cheesiest-dataset  World

In [ ]:
!kaggle datasets download -d astraszab/facial-expression-dataset-image-folders-fer2013

facial-expression-dataset-image-folders-fer2013.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip facial-expression-dataset-image-folders-fer2013.zip

Archive:  facial-expression-dataset-image-folders-fer2013.zip
replace data/test/0/32298.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [162]:
from PIL import Image

im = Image.open('data/train/0/0.png')
# width, height = im.size
print(im.size)

(48, 48)


In [163]:
import os
import itertools # To join a list of lists
import matplotlib.pyplot as plt # For general plotting
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as transforms


# Utility to visualize PyTorch network and shapes
from torchsummary import summary
np.set_printoptions(suppress=True)

# Set seed to generate reproducible "pseudo-randomness" (handles scipy's "randomness" too)
# random_seed = 7
# np.random.seed(random_seed)
# torch.manual_seed(7)

plt.rc('font', size=22)          # controls default text sizes
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=18)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
plt.rc('legend', fontsize=16)    # legend fontsize
plt.rc('figure', titlesize=22)   # fontsize of the figure title

In [164]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define how transform the data for processing in batches
transform = transforms.Compose([
        # transforms.Resize([280,280]), # Resizing the image
        transforms.Grayscale(num_output_channels =1),
        transforms.ToTensor(),
        
        transforms.Normalize(mean=0.5,std=0.5) # 0.5 due to how torchvision data is in range [0,1]
    ])
# Specify the batch size for SGD
batch_size = 64

# Use the ImageFolder class, which is a generic dataloder useful when images stored in labelled directories
# Apply transform to images specified above, importantly converting "ToTensor"
train_dataset = torchvision.datasets.ImageFolder(root='data/train', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root='data/test', transform=transform)
valid_dataset = torchvision.datasets.ImageFolder(root='data/val', transform=transform)

classes = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
print("Train dataset has N = {} samples".format(len(train_dataset)))
print("Test dataset has N = {} samples".format(len(test_dataset)))
print("Validation dataset has N = {} samples".format(len(valid_dataset)))

# Define iterable for our newly created dataset and shuffle samples 
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

### Plotting stuff for visualization ###
fig = plt.figure(figsize=(20,20))
# Utility function to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    # img_np = img.cpu().numpy()
    print(img[0].shape)
    print(img[1].shape)
    print(img[2].shape)
    print(img.shape)
    imshow(np.transpose(img[0].cpu().detach().numpy(), (1, 2, 0)))#np.transpose(img_np, (1, 2, 0))


# Get some random training dataset images
dataiter = iter(train_dataloader)
# print("detailer shape",dataiter.shape)
# Extract a batch
images, labels = dataiter.next()
# images, labels = images.to(device), labels.to(device)
print(images[:8].shape)
# Show 8 images for display
# num_display = 8
# make_grid= torchvision.utils.make_grid(images[:num_display], num_rows=1)
# print("make grid size",make_grid.shape)
# print("slice", make_grid[0].shape)
# imshow(make_grid)
# # Print labels as a concatenated string
# print("\nLabels corresponding to randomly drawn images from the training set:")
# print(" ".join(f'{classes[labels[j]]:5s}\t' for j in range(num_display)))

Train dataset has N = 28709 samples
Test dataset has N = 3589 samples
Validation dataset has N = 3589 samples
torch.Size([8, 1, 48, 48])


<Figure size 1440x1440 with 0 Axes>

In [165]:
print(images.shape[1])

1


In [184]:
# Note that PyTorch uses NCHW (samples, channels, height, width) image format convention
class ConvNet(nn.Module):
    def __init__(self, in_channels, num_filters, out_classes, kernel_size=3):
        super().__init__()
        # Conv2D layer with 'same' padding so image retains shape
        self.conv1 = nn.Conv2d(in_channels, num_filters, kernel_size, padding='same')
        # self.conv2_bn = nn.BatchNorm2d(num_filters)
        self.drop = nn.Dropout(p=0.25)
        self.drop2 =nn.Dropout(p=0.2)
        #self.dropout = 
        # MaxPooling layer with 2x2 kernel size and stride 2
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(num_filters, num_filters*2, kernel_size, padding='same')
        self.conv2_bn = nn.BatchNorm2d(num_filters*2)
        self.conv3 = nn.Conv2d(num_filters*2, num_filters*4, kernel_size, padding='same')
        self.conv3_bn = nn.BatchNorm2d(num_filters*4)
        self.conv4 = nn.Conv2d(num_filters*4, num_filters*8, kernel_size, padding='same')
        self.conv4_bn = nn.BatchNorm2d(num_filters*8)
        self.conv5 = nn.Conv2d(num_filters*8, num_filters*16, kernel_size, padding='same')
        self.conv5_bn = nn.BatchNorm2d(num_filters*16)
        # self.conv6 = nn.Conv2d(num_filters*16, num_filters*32, kernel_size, padding='same')
        # self.conv6_bn = nn.BatchNorm2d(num_filters*32)
        self.fc1 = nn.Linear(num_filters*16*1*1, num_filters)
        self.fc1_norm= nn.BatchNorm1d(num_filters)
        self.fc2 = nn.Linear(num_filters, num_filters)
        self.fc2_norm= nn.BatchNorm1d(num_filters)
        self.fc3 = nn.Linear(num_filters, out_classes)


    def forward(self, x):
        # Non-linear ReLU activations between convolutional layers
        # Conv->ReLU->Pooling
        # 280x280 image -> 140x140 after pooling
        # print("x shape is: ", x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        # print(x.shape)
        # 140x140 feature map -> 70x70 after pooling
        x = self.drop(self.pool(self.conv2_bn(F.relu(self.conv2(x)))))
        # print(x.shape)
        # Flatten all dimensions except batch (start_dim=1)
        x = self.drop(self.pool(self.conv3_bn(F.relu(self.conv3(x)))))
        # print(x.shape)
        x = self.drop(self.pool(self.conv4_bn(F.relu(self.conv4(x)))))
        x = self.drop(self.pool(self.conv5_bn(F.relu(self.conv5(x)))))
        # x = self.drop(self.pool(self.conv6_bn(F.relu(self.conv6(x)))))
        # print("x before flat",x.shape)
        x = torch.flatten(x, 1) 
        # print(x.shape)
        x = self.fc1_norm(F.relu(self.drop(self.fc1(x))))
        x = self.fc2_norm(F.relu(self.drop(self.fc2(x))))
        x = self.fc3(x)
        # print("shape is: ", self.conv1.size, self.conv2.size, self.conv3.size, self.fc1.size, self.fc2.size)
        
        return x

input_channels = images.shape[1]
model = ConvNet(input_channels, num_filters=64, out_classes=len(classes))

# Dummy inputs so we can plot a summary of the neural network's architecture and no. of parameters
model=model.to(device)
summary(model, input_size=(input_channels, 48, 48))

# Double check if model exists
# if os.path.exists(os.getcwd() + '/cnn_fmd.pth'):
#     model.load_state_dict(torch.load(os.getcwd() + '/cnn_fmd.pth'))
#     print("Loaded model from disk!")


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 48, 48]             640
         MaxPool2d-2           [-1, 64, 24, 24]               0
            Conv2d-3          [-1, 128, 24, 24]          73,856
       BatchNorm2d-4          [-1, 128, 24, 24]             256
         MaxPool2d-5          [-1, 128, 12, 12]               0
           Dropout-6          [-1, 128, 12, 12]               0
            Conv2d-7          [-1, 256, 12, 12]         295,168
       BatchNorm2d-8          [-1, 256, 12, 12]             512
         MaxPool2d-9            [-1, 256, 6, 6]               0
          Dropout-10            [-1, 256, 6, 6]               0
           Conv2d-11            [-1, 512, 6, 6]       1,180,160
      BatchNorm2d-12            [-1, 512, 6, 6]           1,024
        MaxPool2d-13            [-1, 512, 3, 3]               0
          Dropout-14            [-1, 51

In [ ]:
def model_train_loader(model, dataloader, criterion, optimizer):
    size = len(dataloader.dataset)

    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction error
        X=X.to(device)
        y=y.to(device)
        predictions = model(X)
        loss = criterion(predictions, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Report loss every 10 batches
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
            
def model_test_loader(model, dataloader, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # Tracking test loss (cross-entropy) and correct classification rate (accuracy)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        model.eval()
        for X, y in dataloader:
            X=X.to(device)
            y=y.to(device)
            predictions = model(X)
            test_loss += criterion(predictions, y)
            correct += (predictions.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"\nTest Error\n\tAccuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# The nn.CrossEntropyLoss() loss function automatically performs a log_softmax() to output
criterion = nn.CrossEntropyLoss()
# One is enough for my toy example...
num_epochs = 200
optimizer = torch.optim.Adam(model.parameters(), lr=0.00008)#, momentum=0.9, nesterov=True)

for t in range(num_epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    model_train_loader(model, train_dataloader, criterion, optimizer)
    model_test_loader(model, test_dataloader, criterion)
    
# Saving the model file 'cnn_fmd.pth' to my current working directory (cwd)
print("Saving model to disk!")
torch.save(model.state_dict(), os.getcwd() + '/cnn_fmd.pth')
# At filter size 64:
#0.000008 epoch- 190 63-64% max   
#0.00001 epoch - 110 62-64% max
#0.00003 epoch - 60 63-64% max
#0.00004 epoch - 40 63-64% max
#0.00006 epoch - 30 63.5-64.3%
#0.00008 epoch - 45 64.7%
#0.0001

Epoch 1
-------------------------------
loss: 2.145180  [    0/28709]
loss: 1.993859  [  640/28709]
loss: 2.034746  [ 1280/28709]
loss: 1.890343  [ 1920/28709]
loss: 1.997593  [ 2560/28709]
loss: 1.977610  [ 3200/28709]
loss: 2.031662  [ 3840/28709]
loss: 1.963867  [ 4480/28709]
loss: 2.154509  [ 5120/28709]
loss: 1.926085  [ 5760/28709]
loss: 1.961634  [ 6400/28709]
loss: 2.105683  [ 7040/28709]
loss: 2.028382  [ 7680/28709]
loss: 2.020552  [ 8320/28709]
loss: 1.950358  [ 8960/28709]
loss: 1.881876  [ 9600/28709]
loss: 1.917986  [10240/28709]
loss: 2.022004  [10880/28709]
loss: 1.949697  [11520/28709]
loss: 2.045978  [12160/28709]
loss: 2.036646  [12800/28709]
loss: 1.964605  [13440/28709]
loss: 1.781789  [14080/28709]
loss: 1.836921  [14720/28709]
loss: 1.940383  [15360/28709]
loss: 1.927135  [16000/28709]
loss: 1.951290  [16640/28709]
loss: 2.016847  [17280/28709]
loss: 1.960751  [17920/28709]
loss: 1.923481  [18560/28709]
loss: 1.873003  [19200/28709]
loss: 1.952561  [19840/28709]


In [ ]:
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.applications import VGG16, InceptionResNetV2
from keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax


model= tf.keras.models.Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(48, 48,1)))
model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
    
model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten()) 
model.add(Dense(256,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
    
model.add(Dense(512,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

model.compile(
    optimizer = Adam(lr=0.0001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
  )

In [ ]:
print(images.shape)

In [ ]:
### Plotting stuff ###
fig = plt.figure(figsize=(20,20))

# Get some random test dataset images
dataiter = iter(test_dataloader)
# Extract a batch
images, labels = dataiter.next()
# Show images
imshow(torchvision.utils.make_grid(images[:num_display]))

outputs = model(images)
# Argmax take the most probable class
_, predicted = torch.max(outputs, 1)

# Print labels
print("\nGroundTruth:\n", " ".join(f'{classes[labels[j]]:5s}\t' for j in range(8)))

print("\nPredicted:\n", " ".join(f'{classes[predicted[j]]:5s}\t' for j in range(8)))

In [ ]:
correct = 0

predicted_list = []
labels_list = []

# Since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    i = 0
    for data in test_dataloader:
        i += 1
        images, labels = data
        # Forward pass through network
        outputs = model(images)
        # Take most probable class
        _, predicted = torch.max(outputs.data, 1)
        predicted_list.append(predicted.detach().numpy())
        labels_list.append(labels.detach().numpy())
        
# Hacky conversions from a list of lists of arrays into joined numpy arrays
predicted_list = np.array(list(itertools.chain.from_iterable(predicted_list)))
labels_list = np.array(list(itertools.chain.from_iterable(labels_list)))
# Count up number of labels and correct predictions
correct = sum(predicted_list == labels_list)

print(f"Accuracy of the network on the test set images: {100 * correct // len(test_dataloader.dataset)} %")

print("Confusion Matrix (columns: True class, rows: Predicted class):")
conf_mat = confusion_matrix(labels_list, predicted_list);
conf_display = ConfusionMatrixDisplay.from_predictions(labels_list, predicted_list, display_labels=classes, colorbar=False);
plt.xlabel("True Labels");
plt.ylabel("Predicted Labels");